In [1]:
import sys
sys.path.insert(0, "../scripts")
import vavg_indoor_air as v
import wall_cond as w
import helpers as h

from sympy import * 
from spb import *

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly as plotly

from icecream import ic

In [2]:
v1 = v.VavgIndoorAir(v.PhysicalConstants(), n_mins=60*24) #extend to 1 day 

In [3]:
times, temps = v1.calc_transient_indoor_air()

In [4]:
def make_colors(min_interv):
    n_colors = len([ix_time for ix_time,_ in enumerate(v1.all_wall_temps) if ix_time%(min_interv*4)==0]) + 1

    colors = px.colors.sample_colorscale("aggrnyl", [n/(n_colors -1) for n in range(n_colors)])

    colors2 = px.colors.sample_colorscale("agsunset", [n/(n_colors -1) for n in range(n_colors)])

    # print(len(colors))

    return colors, colors2


In [5]:
# whether or not can turn time series off and on 
lg = True
fig = make_subplots(rows=1, cols=3, shared_yaxes=False, subplot_titles=("Amb Temp", "Wall Temps", "Room Temp"))

#TODO relate "4" to number of time steps argument in VAvgIndoorAir function 

min_interv = 2*60 # interval to plot at in minutes 
colors, colors2 = make_colors(min_interv=min_interv)

cix=0
for ix_time, x_at_time in enumerate(v1.all_wall_temps): # NEED way to abstract this 
    if ix_time%(min_interv*4)== 0:
        timestamp = [(pd.Timedelta(times[ix_time], "m") + v1.twall.ext_temps.index[0]).strftime('%I %p') ]

        legendgroup = "Same" if lg == False else timestamp[0]

        # Outdoor Air 
        fig.add_trace(go.Scatter(x=timestamp, y=[v1.twall.ext_temps[ix_time]], mode='lines+markers', line=dict(color=colors[cix]),
        showlegend=False, name=timestamp[0], legendgroup=legendgroup
        ), row = 1, col = 1)

        # Wall Temperature 
        fig.add_trace(go.Scatter(x=v1.twall.x_vals, y=x_at_time, mode='lines+markers', line=dict(color=colors[cix]),
        showlegend=True, name=timestamp[0], legendgroup=legendgroup
        ), row = 1, col = 2)

        # Indoor Air 
        fig.add_trace(go.Scatter(x=timestamp, y=[temps[ix_time]], mode='lines+markers', line=dict(color=colors[cix]),
        showlegend=False, name=timestamp[0], legendgroup=legendgroup
        ), row = 1, col = 3)



        cix+=1

fig['layout']['yaxis']['title']='Temperature (K )'
fig['layout']['xaxis1']['title']='Time'
fig['layout']['xaxis2']['title']='Wall Nodes Distance'
fig['layout']['xaxis3']['title']='Time'
fig.show()

In [6]:
fig = go.Figure()

fig = make_subplots(rows=1, cols=2, shared_yaxes=False, subplot_titles=("Unnormalized", "Normalized"))

ix_times = [ix_time for ix_time,_ in enumerate(v1.all_wall_temps) if ix_time%(min_interv*4)==0]
timestamps = [(pd.Timedelta(times[i], "m") + v1.twall.ext_temps.index[0]).strftime('%I %p') for i in ix_times]


fig.add_trace(go.Scatter(x=timestamps, y=temps[ix_times], mode='markers+lines', name="Simulation", line=dict(color="blue")), row = 1, col = 1)
fig.add_trace(go.Scatter(x=timestamps, y=v1.twall.real_temps[ix_times], mode='markers+lines', name="Real", 
line=dict(color="red")
 ), row = 1, col = 1)


fig.add_trace(go.Scatter(x=timestamps, y=h.normalize(temps[ix_times]), mode='markers+lines', name="Simulation", line=dict(color="blue"), showlegend=False), row = 1, col = 2)
fig.add_trace(go.Scatter(x=timestamps, y=h.normalize(v1.twall.real_temps[ix_times]),23 mode='markers+lines', name="Real",
line=dict(color="red"), showlegend=False
 ), row = 1, col = 2)

fig.update_layout(xaxis_title='Time',
                  yaxis_title='Temperature',
                  title='Comparison of Room Temperatures')


In [7]:
fig = go.Figure()

ix_times = [ix_time for ix_time,_ in enumerate(v1.all_wall_temps) if ix_time%(min_interv*4)==0]
timestamps = [(pd.Timedelta(times[i], "m") + v1.twall.ext_temps.index[0]).strftime('%I %p') for i in ix_times]

fig.add_trace(go.Scatter(x=timestamps, y=v1.wall_heat_flux[ix_times], mode='markers+lines',))
fig.update_layout(xaxis_title='Time',
                  yaxis_title='Heat Flux',
                  title='Heat Transfer from Wall to Room')



### plot on same scale!

In [8]:
# whether or not can turn time series off and on 
lg = True
fig = make_subplots(rows=1, cols=3, shared_yaxes=True, subplot_titles=("Amb Temp", "Wall Temps", "Room Temp"))

#TODO relate "4" to number of time steps argument in VAvgIndoorAir function 

min_interv = 2*60 # interval to plot at in minutes 
colors, colors2 = make_colors(min_interv=min_interv)

cix=0
for ix_time, x_at_time in enumerate(v1.all_wall_temps): # NEED way to abstract this 
    if ix_time%(min_interv*4)== 0:
        timestamp = [(pd.Timedelta(times[ix_time], "m") + v1.twall.ext_temps.index[0]).strftime('%I %p') ]

        legendgroup = "Same" if lg == False else timestamp[0]

        # Outdoor Air 
        fig.add_trace(go.Scatter(x=timestamp, y=[v1.twall.ext_temps[ix_time]], mode='lines+markers', line=dict(color=colors[cix]),
        showlegend=False, name=timestamp[0], legendgroup=legendgroup
        ), row = 1, col = 1)

        # Wall Temperature 
        fig.add_trace(go.Scatter(x=v1.twall.x_vals, y=x_at_time, mode='lines+markers', line=dict(color=colors[cix]),
        showlegend=True, name=timestamp[0], legendgroup=legendgroup
        ), row = 1, col = 2)

        # Indoor Air 
        fig.add_trace(go.Scatter(x=timestamp, y=[temps[ix_time]], mode='lines+markers', line=dict(color=colors[cix]),
        showlegend=False, name=timestamp[0], legendgroup=legendgroup
        ), row = 1, col = 3)
        cix+=1

fig['layout']['yaxis']['title']='Temperature (K )'
fig['layout']['xaxis1']['title']='Time'
fig['layout']['xaxis2']['title']='Wall Nodes Distance'
fig['layout']['xaxis3']['title']='Time'
fig.show()